In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [5]:
folder_path = '../data'

In [6]:
def extract_utterance_vad(path, threshold=0.75):

    _vap_result = pd.read_json(path)
    vad = np.array(_vap_result['vad'][0])
    vad_partner, vad_self = vad[:,0],  vad[:,1]
    t = np.linspace(0, (len(vad_self) - 1) * (1/50), len(vad_self))
    
    u_self = np.zeros(len(vad_self))
    u_partner = np.zeros(len(vad_partner))
    u_self[vad_self>threshold] = 1
    u_partner[vad_partner>threshold] = 1

    d_u_self = np.diff(u_self, prepend=[0])
    d_u_partner = np.diff(u_partner, prepend=[0])

    u_self_start, u_self_end = t[d_u_self>=1-1e-3], t[d_u_self<=-1+1e-3]
    if (len(u_self_start)>len(u_self_end)):
        u_self_end = np.append(u_self_end, t[-1]) 
    if (u_self_end[0] < u_self_start[0]):   u_self_end = u_self_end[1:]
    if (u_self_end[-1] < u_self_start[-1]): u_self_start = u_self_start[:-1]

    u_partner_start, u_partner_end = t[d_u_partner>=1-1e-3], t[d_u_partner<=-1+1e-3]
    if (len(u_partner_start)>len(u_partner_end)):
        u_partner_end = np.append(u_partner_end, t[-1])
    if (u_partner_end[0] < u_partner_start[0]):   u_partner_end = u_partner_end[1:]
    if (u_partner_end[-1] < u_partner_start[-1]): u_partner_start = u_partner_start[:-1]

    utter_self = pd.DataFrame({'start': u_self_start, 'end': u_self_end})
    utter_partner = pd.DataFrame({'start': u_partner_start, 'end': u_partner_end})

    return utter_self, utter_partner

In [7]:
# dyad 0-3 --> experiment implementation error
# dyad 7   --> hardware issue

for i in range(4,31):
    if (i==7):
        continue
    
    for p in ['a', 'b']:
        
        pid = f'{i}{p}'

        # voice activity projection
        vad_self, vad_partner = extract_utterance_vad(f'{folder_path}/VadResult/{pid}_stereo.json', threshold=0.75)
        vad_self.to_csv(f"_utterances/{pid}_vad_self.csv")
        vad_partner.to_csv(f"_utterances/{pid}_vad_partner.csv")
